Import libraries

In [1]:
import os

import pandas as pd

import numpy as np

from scipy import stats
import scipy.stats as sc
from scipy.cluster.hierarchy import linkage, leaves_list

import statsmodels.api as sm

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

Import data

In [2]:
dataFolderName = '../data_luc/video/'

listVideoName =   ['x264-1908-bridgefar-wasm/x264-results1.csv', 
                   'x264-1908-ice-wasm/x264-results1.csv',
                   'x264-1908-flower-wasm/x264-results1.csv', 
                   'x264-1908-caire-wasm/x264-results1.csv',
                   'x264-0208-sintel-calda-wasm/x264-results1.csv', 
                   'x264-1908-footballcif-wasm/x264-results1.csv',
                   'x264-0308-crowd_run-wasm/x264-results1.csv', 
                   'x264-0608-blue-wasm/x264-results1.csv',
                   'x264-0608-people-wasm/x264-results1.csv', 
                   'x264-1908-sunflowers-wasm/x264-results1.csv',
                   'x264-0408-deadline-wasm/x264-results1.csv', 
                   'x264-2108-bridgeclose-wasm/x264-results1.csv',
                   'x264-1908-husky-wasm/x264-results1.csv', 
                   'x264-1908-tennis-wasm/x264-results1.csv',
                   'x264-1908-riverbed-wasm/x264-results1.csv', 
                   'x264-0608-park-wasm/x264-results1.csv',
                   'x264-0508-soccer-wasm/x264-results1.csv']

# creation of the list of videos (for each video: x264 configurations + measurements)
listVideo = []

for vn in listVideoName:
    listVideo.append(pd.read_csv(open(dataFolderName+vn,"r")))
    
nbVideos = len(listVideo)

# test
print("There are " + str(len(listVideo)) + " videos")
assert (len(listVideoName) == len(listVideo))


#print a csv
listVideo[7].drop(['no_8x8dct', 'no_deblock', 'no_mixed_refs', 'no_weightb', 'size', 'usertime', 'systemtime', 'H264', 'no_asm', 'configurationID'], axis=1)[:10]

There are 17 videos


,no_cabac,no_fast_pskip,no_mbtree,rc_lookahead,ref,elapsedtime
0,False,True,False,20,9,5.074625
1,True,True,False,40,9,5.902625
2,False,False,True,40,1,2.908125
3,True,True,True,40,9,5.027500
4,False,False,False,60,5,4.329125
5,True,True,False,60,5,4.710125
6,False,False,False,60,1,3.624125
7,True,True,False,60,1,3.680875
8,False,True,False,60,1,3.512125
9,False,True,False,60,9,6.309250


## No transfer implementation

In [3]:
nbVideos = len(listVideo)


def mape(y_true, y_pred):
    return np.mean(np.abs((y_true-y_pred)/y_true))

nbConfig = [16,32,64,128,256,320,448,640,960]

#pct_test_val = [k/10 for k in np.arange(1,10,1)]

res_no_transfer = pd.DataFrame({'id_video' : range(nbVideos)})

mape_no_transfer = np.ones(nbVideos)

nb_runs = 5

for conf in nbConfig:
    for id_video in range(nbVideos):
        
        res_accuracy = []
        
        for i in range(nb_runs):
            
            pct_test = 1-conf/listVideo[i].size

            random_state = np.random.randint(0,1000)

            target = listVideo[id_video] #j

            X_tgt = target[['no_8x8dct','no_asm', 'no_cabac','no_deblock','no_fast_pskip',
                'no_mbtree','no_mixed_refs','no_weightb','rc_lookahead','ref']]

            y_tgt = np.array(target['elapsedtime'], dtype=float)

            X_tgt_train, X_tgt_test, y_tgt_train, y_tgt_test = train_test_split(X_tgt, 
                                                                                y_tgt, 
                                                                                test_size=pct_test, 
                                                                                random_state=random_state)
            clf = RandomForestRegressor()

            clf.fit(X_tgt_train, y_tgt_train)

            y_tgt_pred_test = clf.predict(X_tgt_test)
            
            res_accuracy.append(mape(y_tgt_test, y_tgt_pred_test))
            

        mape_no_transfer[id_video] = min(np.mean(res_accuracy),1)

    res_no_transfer[str(conf)] = mape_no_transfer

In [4]:
res = res_no_transfer.set_index('id_video')
res.to_csv("../data_luc/expe/res_no_transfer_nb_conf_training.csv")
res

,16,32,64,128,256,320,448,640,960
id_video,,,,,,,,,
0,0.292185,0.229803,0.210644,0.213299,0.135309,0.117371,0.089852,0.057866,0.039893
1,0.330666,0.237239,0.206697,0.169935,0.140762,0.105522,0.104436,0.073883,0.050691
2,0.258275,0.217035,0.172817,0.144125,0.143510,0.113450,0.105774,0.067542,0.048117
3,0.354441,0.312414,0.200233,0.208136,0.132765,0.095059,0.073160,0.066341,0.058217
4,0.286318,0.218326,0.174242,0.133535,0.105156,0.099230,0.057287,0.047672,0.031767
5,0.346452,0.205459,0.207902,0.196522,0.130807,0.096993,0.073568,0.048280,0.036160
6,0.342234,0.284874,0.217846,0.159843,0.073666,0.058462,0.046918,0.032151,0.022877
7,0.342865,0.184729,0.188256,0.111767,0.070209,0.054159,0.044593,0.034226,0.026863
8,0.291553,0.221550,0.163249,0.118215,0.077499,0.062904,0.060665,0.047320,0.035680
